# Aula Teórica 4 (guião)
### Semana de 6 a 10 de Outubro 2025
### José Carlos Ramalho
### Sinopsis:

* Conversão de AFND em AFD;
* Análise Léxica.

## Análise de Léxica 

Resumindo, é um bloco que recebe uma lista de carateres (`[Char]`) e produz uma lista de tokens (`[Token]`).

Vamos exemplificar com a linguagem dos parentesis.

Exemplos de frases corretas:

``` 
()
(())()
((()()())())(())
```

Exemplos de frases incorretas:

```
)
(()))
())()
(())()(()
```

Independentemente da correção da frase, a partir da entrada queremos obter uma lista diferente:

```
In: (())()
Out: PA PA PF PF PA PF
```

### Implementação: como fazer?

In [ ]:
# Consumir o input
import sys

for linha in sys.stdin:
    for tok in tokenize(linha):
        print(tok)  

In [ ]:
# Definir a função tokenize
import re

# Tokens a reconhecer: PA, PF, SKIP, NEWLINE, ERRO

def tokenize(input_string):
    reconhecidos = []
    linha = 1
    mo = re.finditer(r'(?P<PA>\()|(?P<PF>\))|(?P<SKIP>[ \t])|(?P<NEWLINE>\n)|(?P<ERRO>.)', input_string)
    for m in mo:
        dic = m.groupdict()
        if dic['PA']:
            t = ("PA", dic['PA'], linha, m.span())

        elif dic['PF']:
            t = ("PF", dic['PF'], linha, m.span())
    
        elif dic['SKIP']:
            t = ("SKIP", dic['SKIP'], linha, m.span())
    
        elif dic['NEWLINE']:
            t = ("NEWLINE", dic['NEWLINE'], linha, m.span())
    
        elif dic['ERRO']:
            t = ("ERRO", dic['ERRO'], linha, m.span())
    
        else:
            t = ("UNKNOWN", m.group(), linha, m.span())
        if not dic['SKIP'] and t[0] != 'UNKNOWN': reconhecidos.append(t)
    return reconhecidos

#### Teste:

```
$ python3 analex_par.py
(())
('PA', '(', 1, (0, 1))
('PA', '(', 1, (1, 2))
('PF', ')', 1, (2, 3))
('PF', ')', 1, (3, 4))
('NEWLINE', '\n', 1, (4, 5))
```

### Mais um exemplo: reconhecer sequências de dígitos no texto de entrada 

```
In: Olha, vai às compras e compra: 2 peras, 4 bananas e 5 tangerinas.
Out: INT INT INT
```

In [ ]:
import sys
import re

def tokenize(input_string):
    reconhecidos = []
    linha = 1
    mo = re.finditer(r'(?P<INT>\d+)|(?P<SKIP>[ \t]|.)|(?P<NEWLINE>\n)|(?P<ERRO>.)', input_string)
    for m in mo:
        dic = m.groupdict()
        if dic['INT']:
            t = ("INT", dic['INT'], linha, m.span())
    
        elif dic['SKIP']:
            t = ("SKIP", dic['SKIP'], linha, m.span())
    
        elif dic['NEWLINE']:
            t = ("NEWLINE", dic['NEWLINE'], linha, m.span())
    
        elif dic['ERRO']:
            t = ("ERRO", dic['ERRO'], linha, m.span())
    
        else:
            t = ("UNKNOWN", m.group(), linha, m.span())
        if not dic['SKIP'] and t[0] != 'UNKNOWN': reconhecidos.append(t)
    return reconhecidos

for linha in sys.stdin:
    for tok in tokenize(linha):
        print(tok)    

```
python3 analex_int.py
Olha, vai às compras e compra: 2 peras, 4 bananas e 5 tangerinas.
('INT', '2', 1, (31, 32))
('INT', '4', 1, (40, 41))
('INT', '5', 1, (52, 53))
('NEWLINE', '\n', 1, (65, 66))
```

## Generalização

O processo é sempre o mesmo pelo que pode ser generalizado bastando isolar a definição dos tokens num ficheiro à parte.

Optou-se por usar um ficheiro JSON para isso (devido ao formato teremos de usar strings normais).

A seguir demonstra-se o processo com o exemplo das expressões aritméticas.

### Definição dos tokens

```json
[
    {
        "id": "ADD",
        "expreg": "\\+"
    },
    {
        "id": "SUB",
        "expreg": "\\-"
    },
    {
        "id": "MUL",
        "expreg": "\\*"
    },
    {
        "id": "DIV",
        "expreg": "\\/*"
    },
    {
        "id": "INT",
        "expreg": "\\d+"
    },
    {
        "id": "SKIP",
        "expreg": "[ \\t]"
    },
    {
        "id": "NEWLINE",
        "expreg": "\\n"
    },
    {
        "id": "ERRO",
        "expreg": "."
    }
]
```

### O Gerador de Analisadores Léxicos

In [ ]:
import sys
import json

def main():
    # Ensure a filename was provided as a command-line argument
    if len(sys.argv) < 2:
        print("Usage: python gen_tokenizer2.py <filename>")
        sys.exit(1)

    filename = sys.argv[1]

    # Try to open and load the specified file
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            tokens = json.load(f)
           
        tokens_regex = '|'.join([f'(?P<{t['id']}>{t['expreg']})' for t in tokens])

        code = f"""
import sys
import re

def tokenize(input_string):
    reconhecidos = []
    linha = 1
    mo = re.finditer(r'{tokens_regex}', input_string)
    for m in mo:
        dic = m.groupdict()
        if dic['{tokens[0]['id']}']:
            t = ("{tokens[0]['id']}", dic['{tokens[0]['id']}'], linha, m.span())
"""

        for t in tokens[1:]:
            code += f"""
        elif dic['{t['id']}']:
            t = ("{t['id']}", dic['{t['id']}'], linha, m.span())
    """
        code += f"""
        else:
            t = ("UNKNOWN", m.group(), linha, m.span())
        if not dic['SKIP'] and t[0] != 'UNKNOWN': reconhecidos.append(t)
    return reconhecidos

for linha in sys.stdin:
    for tok in tokenize(linha):
        print(tok)    
"""
        print(code)

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)
    except json.JSONDecodeError:
        print(f"Error: File '{filename}' is not valid JSON.")
        sys.exit(1)

if __name__ == "__main__":
    main()

### Utilização 

Passos para a utilização do gerador:

1. `$ python3 gen_tokenizer2.py tokens_exp.json > analex_exp.py`
2. `$ python3 analex_exp.py`
3. 

```
$ python3 gen_tokenizer2.py tokens_exp.json > analex_exp.py
$ python3 analex_exp.py 
45+7*8
('INT', '45', 1, (0, 2))
('ADD', '+', 1, (2, 3))
('INT', '7', 1, (3, 4))
('MUL', '*', 1, (4, 5))
('INT', '8', 1, (5, 6))
('NEWLINE', '\n', 1, (6, 7))
```